Based on https://bambinos.github.io/bambi/notebooks/circular_regression.html

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.lines import Line2D
from scipy import stats

In [ ]:
az.style.use("arviz-white")

In [ ]:
x = np.linspace(-np.pi, np.pi, 200)
mus = [0.0, 0.0, 0.0, -2.5]
kappas = [0.001, 0.5, 3, 0.5]
for mu, kappa in zip(mus, kappas):
    pdf = stats.vonmises.pdf(x, kappa, loc=mu)
    plt.plot(x, pdf, label=r"$\mu$ = {}, $\kappa$ = {}".format(mu, kappa))
plt.yticks([])
plt.legend(loc=1);

In [ ]:
u = np.linspace(-12, 12, 200)
plt.plot(u, 2 * np.arctan(u))
plt.xlabel("Reals")
plt.ylabel("Radians");

In [ ]:
data = bmb.load_data("periwinkles")
data.head()

In [ ]:
model_vm = bmb.Model("direction ~ distance", data, family="vonmises")
idata_vm = model_vm.fit(include_response_params=True)

model_n = bmb.Model("direction ~ distance", data)
idata_n = model_n.fit(include_response_params=True)

In [ ]:
az.summary(idata_vm, var_names=["~mu"])

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
posterior_mean = bmb.families.link.tan_2(idata_vm.posterior["mu"])
ax[0].plot(data.distance, posterior_mean.mean(("chain", "draw")))
az.plot_hdi(data.distance, posterior_mean, ax=ax[0])

ax[0].plot(data.distance, data.direction, "k.")
ax[0].set_xlabel("Distance travelled (in m)")
ax[0].set_ylabel("Direction of travel (radians)")
ax[0].set_title("VonMises Family")

posterior_mean = idata_n.posterior["mu"]
ax[1].plot(data.distance, posterior_mean.mean(("chain", "draw")))
az.plot_hdi(data.distance, posterior_mean, ax=ax[1])

ax[1].plot(data.distance, data.direction, "k.")
ax[1].set_xlabel("Distance travelled (in m)")
ax[1].set_title("Normal Family");

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax0 = plt.subplot(121)
ax1 = plt.subplot(122, projection="polar")

model_vm.predict(idata_vm, kind="response")
pp_samples = az.extract_dataset(
    idata_vm, group="posterior_predictive", num_samples=200
)["direction"]
colors = ["C0", "k", "C1"]

for ax, circ in zip((ax0, ax1), (False, "radians", colors)):
    for s in pp_samples:
        az.plot_kde(
            s.values,
            plot_kwargs={"color": colors[0], "alpha": 0.25},
            is_circular=circ,
            ax=ax,
        )
    az.plot_kde(
        idata_vm.observed_data["direction"].values,
        plot_kwargs={"color": colors[1], "lw": 3},
        is_circular=circ,
        ax=ax,
    )
    az.plot_kde(
        idata_vm.posterior_predictive["direction"].values,
        plot_kwargs={"color": colors[2], "ls": "--", "lw": 3},
        is_circular=circ,
        ax=ax,
    )

custom_lines = [Line2D([0], [0], color=c) for c in colors]

ax0.legend(
    custom_lines, ["posterior_predictive", "Observed", "mean posterior predictive"]
)
ax0.set_yticks([])
fig.suptitle("Directions (radians)", fontsize=18);